# Chat Completions API

Chat Completions API는 OpenAI에서 제공하는 대화형 인공지능 모델(GPT 계열)을 활용해, 사용자의 메시지에 대해 자연스러운 대화 응답을 생성하는 API이다. 이 API는 챗봇, AI 비서, 자동화된 상담 시스템 등 다양한 대화형 서비스에 적용할 수 있다.


- **대화 문맥 유지**  
  Chat Completions API는 여러 메시지(대화 내역)를 입력받아, 이전 대화의 맥락을 이해하고 그에 맞는 응답을 생성한다. 즉, 단순히 한 문장만을 이어 쓰는 것이 아니라, 대화의 흐름을 반영하여 자연스러운 대화를 이어갈 수 있다.

- **역할(Role) 기반 메시지 구조**  
  입력 메시지는 배열 형태로 전달하며, 각 메시지는 `role`과 `content`로 구성된다.  
  - `system`: AI의 태도, 성격, 역할을 정의(예: "너는 친절한 도우미야.")
  - `user`: 사용자의 질문이나 요청
  - `assistant`: AI의 응답(이전 대화 내용 포함 가능)
  
  이 구조를 통해 AI의 응답 스타일이나 맥락을 세밀하게 제어할 수 있다[1][3][5].

**주요 파라미터 설명**

| 파라미터        | 설명                                                                 |
|----------------|----------------------------------------------------------------------|
| model          | 사용할 언어 모델명 (예: gpt-3.5-turbo, gpt-4o 등)                    |
| messages       | 대화 내역(역할/내용 포함) 배열                                        |
| max_tokens     | 생성할 응답의 최대 토큰 수(선택)                                     |
| temperature    | 창의성 조절(0~2, 낮을수록 일관성↑, 높을수록 다양성↑, 선택)           |
| top_p          | 누적 확률 기반 샘플링(temperature와 유사, 선택)                      |
| n              | 한 번에 생성할 응답 개수(선택)                                       |
| stop           | 응답 생성을 중단할 문자열 목록(선택)                                 |
| presence_penalty, frequency_penalty | 반복 억제 및 창의성 유도(선택)                 |
| user           | 사용자 식별자(선택, abuse monitoring 등 활용)                        |


- 위 예시에서 `messages` 배열에는 대화의 모든 메시지가 순서대로 들어가야 한다.  
- OpenAI는 이전 요청을 기억하지 않기 때문에, 매 API 호출마다 대화 내역 전체를 함께 보내야 한다.

In [7]:
from google.colab import userdata
import os
from openai import OpenAI

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

# 환경 변수 지정
# os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
# client = OpenAI()

## 대화형 챗봇
- single-turn
- multi-turn

single-turn

In [19]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {'role': 'system', 'content': '너는 친절한 챗봇입니다.'},
        {'role': 'user', 'content': '내 이름은 차운우야~~~~~~'},
    ]
)
print(response.choices[0].message.content)

안녕하세요, 차운우님! 반갑습니다! 오늘 기분은 어떤가요?


multi-turn

In [18]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {'role': 'system', 'content': '너는  LLM 전문가입니다.'},
        {'role': 'user', 'content': '안녕 나는 LLM 꿈나무 차운우야~~~~~~'},
        {'role': 'assistant', 'content': '안녕하세요, 차운우님! LLM에 대해 궁금한 점이나 배우고 싶은 내용이 있으면 말씀해 주세요. 함께 이야기해봐요!'},
        {'role': 'user', 'content': 'Transformer 모델을 공부하고 싶어'},
        {'role': 'assistant', 'content': """
        좋은 선택이에요! Transformer 모델은 자연어 처리(NLP) 분야에서 아주 중요한 기술입니다. Transformer는 기본적으로 인코더-디코더 구조를 가지고 있으며, 주로 다음과 같은 요소로 구성되어 있습니다:

        1. **인코더(Encoder)**: 입력 문장을 이해하는 역할을 합니다. 여러 층의 인코더가 서로 병렬로 작동하며, 각 인코더 층은 자기 주의(attention) 메커니즘을 사용해 입력 단어 간의 관계를 파악합니다.

        2. **디코더(Decoder)**: 인코더의 출력을 기반으로 출력 문장을 생성합니다. 디코더 역시 여러 층을 가지며, 이전의 출력과 인코더의 출력을 모두 고려하여 다음 단어를 예측합니다.

        3. **자기 주의(Self-Attention)**: 각 단어가 다른 단어와의 관계를 학습할 수 있도록 도와주는 메커니즘입니다. 이를 통해 문장의 구조와 의미를 잘 캡쳐할 수 있습니다.

        4. **포지셔널 인코딩(Positional Encoding)**: 입력 단어의 순서 정보를 모델에 제공하기 위해 사용됩니다. Transformer는 순차적인 정보를 자연스럽게 처리하지 않기 때문에, 각 단어의 위치 정보를 추가합니다.

        5. **피드 포워드 네트워크(Feed-Forward Networks)**: 각 인코더와 디코더 층 내에서 사용되는 신경망으로, 각 단어의 표현을 변형하고 더 깊은 특징을 추출하는 역할을 합니다.

        Transformer 모델은 특히 BERT, GPT, T5 등과 같은 여러 강력한 NLP 모델의 기반이 됩니다.

        이 모델에 대해 더 깊이 알고 싶은 부분이 있으면 말씀해 주세요!
        """},
        {'role': 'user', 'content': '어려워~ 어텐션을 초등학생도 이해할 수 있게 설명해줘.'},
        {'role': 'assistant', 'content': """
        물론이죠! 어텐션(attention) 개념을 초등학생도 이해할 수 있도록 쉽게 설명해볼게요.

        어텐션은 마치 친구와 대화할 때처럼 생각하면 돼요.

        예를 들어, 당신이 친구와 이야기를 하고 있을 때, 친구의 말 중에서 가장 중요한 부분에 집중하는 것과 같아요. 친구가 "나는 키가 크고, 축구를 좋아해!"라고 말할 때, 우리는 "축구를 좋아해!"라는 부분에 더 주목할 수도 있죠. 이처럼 어떤 정보에 더 집중하고, 다른 정보는 덜 중요하게 여기는 것이 어텐션이에요.

        Transformer 모델에서도 비슷하게 동작해요. 모델이 문장을 이해할 때, 각 단어가 서로에게 얼마나 중요한지를 판단하고, 중요한 단어에 더 많은 "주목"을 하게 해줍니다. 그래서 중요한 정보는 잘 기억하고, 덜 중요한 정보는 덜 기억하게 되는 거예요.

        그렇게 함으로써 모델은 문장의 의미를 더 잘 파악할 수 있게 됩니다. 친구와 대화할 때 중요한 부분에 집중하는 것처럼요!

        이해가 되었나요? 추가적으로 궁금한 점이 있다면 언제든지 물어보세요!

        """},
        {'role': 'user', 'content': '이해가 되는 것 같아. Transformer 내용을 요약해서 하나의 markdown 문서로 정리해줘.'}
    ]
)
print(response.choices[0].message.content)

물론이죠! Transformer 모델에 대한 내용을 요약한 Markdown 문서를 아래에 작성해드릴게요.

```markdown
# Transformer 모델 개요

Transformer는 자연어 처리(NLP)에서 널리 사용되는 모델로, 2017년 Google의 논문 "Attention is All You Need"에서 처음 소개되었습니다. 이 모델은 입력 데이터를 처리하고, 다음 단어를 예측하는 데 강력한 성능을 보여줍니다.

## 구성 요소

1. **인코더(Encoder)**
   - 입력 문장을 처리하여 내부 표현으로 변환.
   - 여러 층의 인코더가 병렬로 작동.
   - 자기 주의(attention) 메커니즘을 사용하여 각 단어 간의 관계를 학습.

2. **디코더(Decoder)**
   - 인코더의 출력을 기반으로 새로운 문장을 생성.
   - 이전 출력과 인코더의 출력을 정보로 사용하여 다음 단어를 예측.

3. **자기 주의(Self-Attention)**
   - 입력의 각 단어가 다른 단어와의 관계를 학습할 수 있도록 도움.
   - 중요한 정보를 더 주목하고 덜 중요한 정보는 덜 주목하게 함.

4. **포지셔널 인코딩(Positional Encoding)**
   - 입력 단어의 순서 정보를 제공하기 위한 기술.
   - Transformer는 순서 정보를 자연스럽게 처리하지 않기 때문에 필요.

5. **피드 포워드 네트워크(Feed-Forward Networks)**
   - 각 층 내에서 단어의 표현을 변형하고 더 깊은 특징을 추출하는 역할.

## 작동 방식

1. 입력 문장은 인코더에 의해 처리되고, 각 단어의 의미가 내재화됨.
2. 디코더는 인코더의 출력을 바탕으로 문장이 생성되며, 이전의 단어들을 토대로 다음 단어를 예측.
3. 자기 주의를 통해 각 단어 간의 연관성을 파악하여 더 자연스럽고 의미 있는 결과를 생성.

## 활용 사례

- BERT, GPT, T5 등과 같은 다양한 자연어 처리 모델의 기반.
- 기계 번역

## 반복처리

In [34]:
# 대화내역을 로깅
messages = [
    {'role': 'system', 'content': '너는 친절한 챗봇이다.'}
]

print('종료하려면, exit를 입력하세요...')
while True:
    # 사용자 입력
    user_input = input('User: ')

    if user_input.strip().lower() == 'exit':
        print('채팅을 종료합니다...')
        break

    # messages에 사용자 입력 추가
    messages.append({'role': 'user', 'content': user_input})

    # LLM 요청
    response = client.chat.completions.create(
        model='gpt-4.1',
        messages=messages,
        temperature=1
    )
    assistant_message = response.choices[0].message.content
    print(f'Assistant: {assistant_message}')

    # messages 챗봇 출력 추가
    messages.append({'role': 'assistant','content': assistant_message})

종료하려면, exit를 입력하세요...
User: exit
채팅을 종료합니다...


## stream

In [32]:
stream = client.chat.completions.create(
  model='gpt-4o-mini',
  messages=[{'role': 'user','content': '지금 stream 테스트할거야.아주 긴 응답 메세지를 보내줘.'}],
  stream=True
)
for chunk in stream:
  content = chunk.choices[0].delta.content
  if content is not None:
    print(content, end='')

물론입니다! 아래는 예시로 제공하는 긴 응답 메시지입니다. 내용을 다양하게 구성해보았습니다.

---

안녕하세요! 오늘 우리는 여러 가지 주제에 대해 이야기해 볼 예정입니다. 우선, 현재 세계에서 가장 주목받고 있는 몇 가지 주제를 소개하고, 각각의 주제에 대해 더 깊이 있는 논의를 진행해 보겠습니다.

첫 번째로, 기후 변화에 대한 논의입니다. 기후 변화는 현재 전 세계가 직면하고 있는 가장 심각한 문제 중 하나로 꼽힙니다. 과학자들은 지구의 평균 기온이 지속적으로 상승하고 있으며, 이는 인간의 활동에 의해 주로 발생한다고 보고하고 있습니다. 특히, 화석 연료의 연소, 임업, 그리고 농업에서 발생하는 온실가스가 주요 원인으로 지목되고 있습니다. 이러한 변화는 생태계의 파괴, 해수면 상승, 기상이변 등 여러 가지 심각한 결과를 초래하고 있으며, 이를 해결하기 위한 국제적인 협력이 필요합니다.

두 번째 주제는 기술 혁신에 관한 것입니다. 인공지능(AI), 빅데이터, 블록체인 등 최신 기술들이 비즈니스와 일상생활을 혁신적으로 변화시키고 있습니다. 특히, 인공지능 기술은 다양한 분야, 예를 들어 의료, 금융, 교육 등에서 활용되고 있으며, 이는 기업의 생산성과 효율성을 높이는 데 크게 기여하고 있습니다. 그러나 이러한 기술의 발전은 동시에 윤리적인 문제를 동반하기도 합니다. AI의 사용이 개인의 프라이버시를 침해할 위험이 있으며, 일자리에 미치는 영향 etc., 이러한 문제들은 우리가 해결해 나가야 할 중요한 과제가 될 것입니다.

세 번째로는 사회적 불평등에 관한 주제입니다. 세계적으로 많은 국가에서 경제적 불평등이 심화되고 있으며, 이는 사회의 안정성과 지속 가능한 발전에 부정적인 영향을 미칠 수 있습니다. 교육, 건강, 기회 등의 불균형이 발생하고 있으며, 이러한 문제를 해결하기 위해서는 정책적인 접근이 필요합니다. 각국 정부는 사회 안전망을 강화하고, 교육 기회를 평등하게 제공하여 사회적 이동성을 증진해야 합니다.

마지막으로, 인류의 미래에 대한 전망입

In [43]:
# stream 버전
messages = [
    {'role': 'system', 'content': '너는 친절한 챗봇이다.'}
]

print('종료하려면, exit를 입력하세요...')
while True:
    # 사용자 입력
    user_input = input('User: ')

    if user_input.strip().lower() == 'exit':
        print('채팅을 종료합니다...')
        break

    # messages에 사용자 입력 추가
    messages.append({'role': 'user', 'content': user_input})

    # LLM 요청
    stream = client.chat.completions.create(
      model='gpt-4o-mini',
      messages=messages,
      temperature=1,
      stream=True
    )
    print(f'Assistant: ', end='')
    for chunk in stream:
      content = chunk.choices[0].delta.content
      if content is not None:
        print(content, end='', flush=True) # 내부 buffer 사용하지 않음
    print()

      # messages 챗봇 출력 추가
    messages.append({'role': 'assistant','content': assistant_message})

종료하려면, exit를 입력하세요...
User: 안녕~
Assistant: 안녕하세요! 어떻게 도와드릴까요?
User: 너 스트림기법으로 대화하는거야?
Assistant: 아니요, 저는 스트림 기법을 사용하지 않고, 사용자의 질문이나 요청에 따라 적절한 답변을 생성하는 방식으로 작동합니다. 대화는 자연어 처리 기술을 통해 이루어지며, 그렇게 해서 최상의 응답을 제공하기 위해 노력하고 있어요. 궁금한 점이 있으면 언제든지 물어보세요!
User: 그럼 스트림은 뭐지?
Assistant: "스트림"이라는 용어는 여러 가지 의미로 사용될 수 있는데, 주로 다음과 같은 맥락에서 쓰입니다:

1. **데이터 스트리밍**: 실시간으로 데이터를 전송하거나 처리하는 방법을 말합니다. 예를 들어, 음악이나 비디오를 웹에서 스트리밍하여 다운로드 없이 바로 재생하는 방식이 이에 해당합니다.

2. **프로그래밍에서의 스트림**: 데이터의 흐름을 의미합니다. 예를 들어, 입력 스트림(Input Stream)은 데이터를 읽어오는 방식이고, 출력 스트림(Output Stream)은 데이터를 내보내는 방식입니다.

3. **대화 형식에서의 스트림**: 챗봇이나 AI 시스템이 사용자와의 대화를 순차적으로 처리하는 방식을 설명할 때 사용됩니다. 즉, 사용자의 입력을 순차적으로 받고, 그에 따른 출력을 순서대로 생성하는 방식입니다.

스트리밍은 대량의 데이터를 효율적으로 처리하고, 사용자 경험을 개선하는 데 매우 유용한 기법입니다. 어떤 맥락에서 스트림에 대해 궁금하신지에 따라 더 자세한 설명을 드릴 수 있습니다!
User: 아 버퍼링을 없애주는게 스트림 맞아? 그 순차적으로 하는거 지금 대화형을 배우다가 스트림을 배웠어
Assistant: 네, 맞습니다! 스트리밍(streaming)이라는 개념은 데이터 또는 정보를 실시간으로 연속적으로 전송하는 것을 의미합니다. 이를 통해 데이터를 한 번에 큰 덩어리로 전송하는 대신, 작은 조각으로 나누어 순차적으로 전송하고, 사용자가 필요할 때 바로바로 

# Token counting
비용 = ((입력토큰수 * 단가) + (출력 토큰수 + 단가)) * 월 서비스 호출 수

In [37]:
!pip install tiktoken

In [46]:
# 각 모델에 따른 토커나이저(인코딩) 가져오기
import tiktoken

gpt35 = tiktoken.encoding_for_model('gpt-3.5')
gpt4o = tiktoken.encoding_for_model('gpt-4o')
# gpt41 = tiktoken.encoding_for_model('gpt-4.1')
gpt41 = tiktoken.get_encoding('cl100k_base')

print(gpt35)
print(gpt4o)
print(gpt41)

<Encoding 'cl100k_base'>
<Encoding 'o200k_base'>
<Encoding 'cl100k_base'>


In [44]:
# 토큰수 세기
encoded_gpt35 = gpt35.encode('아버지가 방에 들어가신다')
encoded_gpt4o = gpt4o.encode('아버지가 방에 들어가신다')

print(len(encoded_gpt35)) # 12
print(len(encoded_gpt4o)) # 9

12
9


## 토큰 비용 계산하기

In [48]:
# 뉴스 기사 스크랩
text = """
더불어민주당이 6일 한동훈 국민의힘 대표가 윤석열 대통령 탄핵에 사실상 찬성 입장을 시사하자 7일로 예정됐던 윤석열 대통령의 탄핵소추안 표결을 앞당기는 방안을 고심하고 있다. 이재명 민주당 대표는 “한 대표의 입장을 정확하게 파악하는 것이 우선”이라며 신중한 태도를 보였다.

이 대표는 이날 기자들을 만나 윤 대통령 탄핵 표결을 앞당기는 방안에 대해 “지금 저렇게 불확실한 얘기를 믿고 미리 당겨서 협의를 할 필요가 있는가, 그런 생각이 일단 든다”고 말했다. 한 대표와의 회동 가능성에 대해서는 “요청은 했는데 아직 결정을 통보받지 못했다. (한 대표 측에서) 오후에 다시 연락하자는 연락이 왔다”고 전했다.

이 대표는 또 “사실 오늘 밤이 저는 매우 위험하다고 생각이 드는데, 제가 가진 감으로 본다면 오늘 밤 새벽에 또 뭔가 일을 벌이지 않을까 그런 걱정이 들긴 한다”며 ‘2차 계엄’ 가능성을 우려했다.

민주당은 이날 오전 한동훈 대표가 “윤 대통령의 조속한 직무 집행 정지가 필요하다”며 입장을 선회하자 긴급 의원총회를 열고 당내 의견 수렴에 나섰다. 의원총회에서는 탄핵소추안 표결 시점을 앞당기는 방안도 논의될 전망이다.

노종면 원내대변인은 비공개 최고위원 간담회가 끝난 뒤 “한 대표의 입장이 보도된 이후 긴장감이 높아지고 있고, 12월 3일 당일에 짐작했던 것 이상으로 치밀하게 의원, 정치인 체포 시도가 있었던 것과 이번 내란 사태에서 매우 중요한 작전이었던 걸로 파악되고 있다”며 “윤 대통령 옹위 세력이 어떻게 나올지 모르는 상황이라고 판단해 이런 비상한 상황 인식 떄문에 긴급 의원총회를 소집했다”고 전했다.

탄핵소추안 표결 시점 변경에 대해서는 “의장실에 본회의 일정 변경을 요청한 바는 아직 없다”며 “일단 신중하고 침착하게 대응할 것이고, 지금 한 대표 쪽의 입장이 뭔지 정확하게 파악하는 것이 우선이다. 필요하면 본회의를 앞당기는 방안도 의장실과 협조해서 추진할 수 있지만 아직은 결정된 바 없다”고 밝혔다.

민주당에서는 7일 오후 7시로 예정됐던 표결을 2시간 당겨 오후 5시에 추진하는 방안도 거론된다. 박성준 원내운영수석부대표는 이날 MBC 라디오 ‘김종배의 시선집중’ 인터뷰에서 “당초 오후 7시 정도 표결을 예상했는데 5시 정도는 해야 한다고 보고 있다”며 “국민의힘에서 탄핵소추안 투표 관련 상당한 지연 전략을 펼쳐서 시간을 늦출 수 있는 상황까지 고려하고 있다”고 설명했다.
"""

encoded_text_gpt41 = gpt41.encode(text)
encoded_text_gpt4o = gpt4o.encode(text)

print('gpt-4o 토큰수:', len(encoded_text_gpt4o))
print('gpt-4.1 토큰수:', len(encoded_text_gpt41))

gpt-4o 토큰수: 703
gpt-4.1 토큰수: 1215


In [51]:
response_gpt4o = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {'role': 'system', 'content': '너는 똑부러지는 시사/경제 전문가로써, 제공된 뉴스기사의 핵심을 잘 요약해서 정리해주는 챗봇이야.'},
        {'role': 'user', 'content': text}
    ],
    temperature=0.1
)

response_gpt41 = client.chat.completions.create(
    model='gpt-4.1',
    messages=[
        {'role': 'system', 'content': '너는 똑부러지는 시사/경제 전문가로써, 제공된 뉴스기사의 핵심을 잘 요약해서 정리해주는 챗봇이야.'},
        {'role': 'user', 'content': text}
    ],
    temperature=0.1
)

output_gpt4o = response_gpt4o.choices[0].message.content
output_gpt41 = response_gpt41.choices[0].message.content

print(output_gpt4o)
print(output_gpt41)

print('\n', 'gpt-4o 토큰수: ', len(gpt4o.encode(output_gpt4o)))
print('\n', 'gpt-41 토큰수: ', len(gpt41.encode(output_gpt41)))

더불어민주당은 한동훈 국민의힘 대표가 윤석열 대통령 탄핵에 사실상 찬성하는 입장을 시사하자, 탄핵소추안 표결 시점을 앞당기는 방안을 검토하고 있습니다. 이재명 민주당 대표는 한 대표의 입장을 정확히 파악하는 것이 우선이라며 신중한 태도를 보였습니다. 민주당은 한 대표의 발언 이후 긴급 의원총회를 열어 당내 의견을 수렴하고 있으며, 표결 시점을 당초 예정된 7일 오후 7시에서 오후 5시로 앞당기는 방안을 논의 중입니다. 이는 국민의힘의 지연 전략을 고려한 조치로 보입니다.
### 기사 핵심 요약

**1. 민주당, 윤석열 대통령 탄핵소추안 표결 시점 앞당길지 고심**
- 한동훈 국민의힘 대표가 윤 대통령 탄핵에 사실상 찬성하는 듯한 입장을 내비치자, 더불어민주당이 7일로 예정된 탄핵소추안 표결을 앞당기는 방안을 논의 중임.
- 이재명 민주당 대표는 한 대표의 진짜 입장을 먼저 정확히 파악해야 한다며 신중한 태도를 보임.

**2. 긴급 의원총회 소집 및 비상 상황 인식**
- 한동훈 대표의 입장 변화 이후 민주당은 긴급 의원총회를 열어 당내 의견을 수렴.
- 민주당은 최근 정치인 체포 시도, 내란 사태 등으로 긴장감이 높아진 상황을 비상하게 인식하고 있음.

**3. 표결 시점 변경 가능성**
- 민주당은 7일 오후 7시로 예정된 표결을 오후 5시로 앞당기는 방안도 검토 중.
- 국민의힘이 표결 지연 전략을 쓸 가능성에 대비해 시간 조정 논의.
- 아직 본회의 일정 변경은 공식 요청하지 않았으며, 한동훈 대표 측 입장 확인 후 결정할 방침.

**4. 이재명 대표의 우려**
- 이 대표는 “오늘 밤이 매우 위험하다”며 2차 계엄 등 추가 돌발 상황을 우려함.

---

**핵심 포인트 한 줄 요약:**  
한동훈 대표의 탄핵 시사 발언 이후 민주당이 윤 대통령 탄핵소추안 표결을 앞당길지 신중히 검토 중이며, 긴급 의원총회 등 비상 대응에 나섬.

 gpt-4o 토큰수:  159

 gpt-41 토큰수:  658


In [54]:
# 모델별 가격(2025년 6월 기준, 1M=1,000,000 토큰)
PRICING = {
    "gpt-4.1": {
        "input": 2.00,    # $2.00 / 1M input tokens
        "output": 8.00    # $8.00 / 1M output tokens
    },
    "gpt-4.1-mini": {
        "input": 0.40,    # $0.40 / 1M input tokens
        "output": 1.60    # $1.60 / 1M output tokens
    },
    "gpt-4.1-nano": {
        "input": 0.10,    # $0.10 / 1M input tokens
        "output": 0.40    # $0.40 / 1M output tokens
    },
    "o1": {
        "input": 2.00,    # $2.00 / 1M input tokens
        "output": 8.00    # $8.00 / 1M output tokens
    },
    "o3": {
        "input": 2.00,    # $2.00 / 1M input tokens
        "output": 8.00    # $8.00 / 1M output tokens
    },
    "o4-mini": {
        "input": 1.10,    # $1.10 / 1M input tokens
        "output": 4.40    # $4.40 / 1M output tokens
    },
    "gpt-4o": {
        "input": 2.50,    # $2.50 / 1M input tokens
        "output": 10.00   # $10.00 / 1M output tokens
    },
    "gpt-4o-mini": {
        "input": 0.15,    # $0.15 / 1M input tokens
        "output": 0.60    # $0.60 / 1M output tokens
    }
}

def count_tokens(text, model):
  if model == 'gpt-4.1':
    encoding = tiktoken.get_encoding('cl100k_base')
  else:
    encoding = tiktoken.encoding_for_model(model)
  encoded = encoding.encode(text)
  return len(encoded)

def calc_cost(input_text, output_text, model, num_service_call=1_000_000):
  input_tokens = count_tokens(input_text, model)
  output_tokens = count_tokens(output_text, model)

  # 모델별 단가 가져오기
  price = PRICING[model]

  # 비용계산
  input_cost = (input_tokens / 1_000_000) * price['input']
  output_cost = (output_tokens / 1_000_000) * price['output']

  total_cost = (input_cost + output_cost) * num_service_call
  return total_cost

print('$', calc_cost(text, output_gpt4o, model='gpt-4o-mini'))
print('$', calc_cost(text, output_gpt41, model='gpt-4.1'))

$ 200.84999999999997
$ 7693.999999999999
